# Necessary to install Python 3.11 or 3.10 in SageMaker-Studio-Lab


In [ ]:
conda create --name sdpy311env python=3.11 -y
conda activate sdpy311env
pip install ipykernel
python -m ipykernel install --user --name sdpy311env --display-name "Python 3.11 (sdpy311env)"


# Python dependencies!

In [ ]:
from IPython.display import clear_output

#Conda install and update
!conda install -c conda-forge libglib -y
!conda update -n base conda -y
#Conda installation for PyGObject with automatic "yes" confirmation
!conda install -c conda-forge pygobject -y
# Pip installation for OpenCV and other packages (no need for -y with pip)
!pip install opencv-python
!pip install opencv-contrib-python
clear_output()
print('[1;32mDone!')

#  Stable-Diffusion dependencies!

In [ ]:
from IPython.display import clear_output
# Install diffusers and other dependencies using pip
!pip install diffusers transformers accelerate scipy safetensors
!pip install torch torchvision --upgrade
!pip install xformers
# insightface for faceswaping optional if you are using
!pip install insightface

clear_output()
print('[1;32mDone!')

# Stable-Diffusion Cloning and Installing Requirements

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git

from IPython.display import clear_output

%cd ~/stable-diffusion-webui
!pip install -r requirements.txt

%cd ~/home/studio-lab-user

clear_output()
print('[1;32mDone!')

# Models

In [1]:
import requests
import time
import os
import subprocess

# Download path
download_path = "/home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/"

# Ensure the download path exists, create it if it doesn't
os.makedirs(download_path, exist_ok=True)

# Helper function to convert bytes into human-readable format
def format_size(size):
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024

# Function to check available storage space
def check_storage_space():
    result = subprocess.run(["df", "-h", "--output=avail,target"], capture_output=True, text=True)
    lines = result.stdout.splitlines()

    for line in lines:
        if '/home' in line:
            available_space = line.split()[0]  # Get available space value
            return available_space
    return "0M"  # Default value if no storage info available

# Function to download model with reduced output frequency
def download_model(url, filename, headers):
    try:
        # Automatically handle the file extension based on the filename provided
        response = requests.get(url, headers=headers, stream=True)

        if response.status_code == 200:
            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0
            start_time = time.time()

            with open(filename, 'wb') as file:
                chunk_size = 1024 * 1024  # Reduce the chunk size
                for chunk in response.iter_content(chunk_size=chunk_size):
                    if chunk:
                        try:
                            file.write(chunk)
                            downloaded_size += len(chunk)

                            if downloaded_size >= 10 * 1024 * 1024:  # Log progress every 10 MB
                                elapsed_time = time.time() - start_time
                                speed = (downloaded_size / (1024 * 1024)) / elapsed_time
                                print(f"Downloaded {format_size(downloaded_size)} of {format_size(total_size)} at {speed:.2f} MB/s", end='\r')
                        except OSError as e:
                            if "No space left on device" in str(e):
                                print(f"\n\nError: No space left on device while downloading {filename}.")
                                return "no_space"  # Return a specific message if no space
                            raise
            print(f"\n\nDownloaded {filename} successfully!\n")
            return "success"
        else:
            print(f"\n\nFailed to download {filename}. Status code: {response.status_code}, Message: {response.text}\n")
            return "error"
    except Exception as e:
        print(f"\n\nError during download of {filename}: {e}\n")
        if "data exceeds" in str(e).lower():
            print("Error: Data exceeds Jupyter memory limit. Please download outside Jupyter Lab.\n")
        return "error"

# Combined function to handle both Hugging Face and CivitAI links
def download_models(model_list, huggingface_headers, civitai_headers):
    for model in model_list:
        if model["enabled"]:
            url = model["link"]
            filename = os.path.join(download_path, model["filename"])

            # Identify if it's Hugging Face or CivitAI by checking the URL structure
            if "huggingface.co" in url:
                headers = huggingface_headers
                platform = "Hugging Face"
            elif "civitai.com" in url:
                headers = civitai_headers
                platform = "CivitAI"
            else:
                print(f"\n\nUnknown platform for {filename}. Skipping...\n")
                continue

            print(f"\nStarting download from {platform} for {filename}...\n")

            # Check storage before downloading
            available_space = check_storage_space()
            print(f"Available space before downloading: {available_space}\n")

            if available_space.endswith('G') or available_space.endswith('M'):
                result = download_model(url, filename, headers)
                if result == "no_space":
                    print(f"\n\nNo space available. Stopping remaining downloads.\n")
                    break
            else:
                print(f"\n\nNo space available for {filename}. Skipping...\n")
        else:
            print(f"\n\nSkipping {model['filename']} (Disabled)\n")





# ----------------------------------------
# Example combined model inputs (both Hugging Face and CivitAI)
model_list = [

    {"link": "https://huggingface.co/ckpt/CyberRealistic/resolve/main/cyberrealistic_v32.safetensors?download=true",
     "filename": "Cyberrealistic.safetensors",     "enabled": True  },

    {"link": "https://civitai.com/api/download/models/143906",
     "filename": "EpiCRealism.safetensors",    "enabled": True  },

    {"link": "https://civitai.com/api/download/models/901206?type=Model&format=SafeTensor&size=full&fp=fp16",
     "filename": "Poetry Dream tassel.safetensors",    "enabled": False  },

    {"link": "https://civitai.com/api/download/model_link_4",
     "filename": "model4",    "enabled": False  }


]



# ----------------------------------------
# Token Section (define them here and create headers below)
huggingface_token = "your_huggingface_token"
civitai_api_key = "your_civitai_token"




# Now, after tokens are defined, create the headers
huggingface_headers = {
    "Authorization": f"Bearer {huggingface_token}"
}
civitai_headers = {
    "Authorization": f"Bearer {civitai_api_key}"
}

# Call the combined download function
download_models(model_list, huggingface_headers, civitai_headers)

# Clear cache after downloads
print("\n\nClearing cache...\n")
subprocess.run(["rm", "-rf", ".cache"])

# Final storage check
print("\nAll downloads done!")
print(f"Available space after downloading: {check_storage_space()}\n")



Starting download from Hugging Face for /home/studio-lab-user/stable-diffusion-webui/models/Stable-diffusion/Cyberrealistic.safetensors...

Available space before downloading: 0M



KeyboardInterrupt: 

# Controal-Net extensions + Controal-Net-Models

In [ ]:
from IPython.display import clear_output
# aria2 for parallel downloads, along with other libraries
!conda install -y -c conda-forge libglib ffmpeg aria2
clear_output()
#Control-Net Extension#
!git clone https://github.com/Mikubill/sd-webui-controlnet /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet




#Control-Net Models#

# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_ip2p_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11e_sd15_shuffle_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_canny_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_depth_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_depth_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_inpaint_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_lineart_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_mlsd_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_normalbae_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_openpose_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_scribble_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_seg_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15_softedge_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/control_v11u_sd15_tile_fp16.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v11u_sd15_tile_fp16.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_style_sd14v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_style_sd14v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd14v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd14v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_seg_sd14v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_seg_sd14v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_openpose_sd14v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_openpose_sd14v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_keypose_sd14v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_keypose_sd14v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_color_sd14v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_color_sd14v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_canny_sd15v2.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_canny_sd15v2.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_depth_sd15v2.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_depth_sd15v2.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_sketch_sd15v2.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_sketch_sd15v2.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/ControlNet-v1-1/resolve/main/t2iadapter_zoedepth_sd15v1.pth -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o t2iadapter_zoedepth_sd15v1.pth
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v1p_sd15_qrcode_monster.safetensors
# !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/v2/control_v1p_sd15_qrcode_monster_v2.safetensors -d /home/studio-lab-user/stable-diffusion-webui/extensions/sd-webui-controlnet/models -o control_v1p_sd15_qrcode_monster_v2.safetensors

print('[1;32mDone!')

!rm -rf /home/studio-lab-user/.cache
!df -h | grep -E 'Avail|home'

print('[1;32mDone!')

# Launch (A1111-WebUi)


In [ ]:
# Replace Ngrok Token with your actual Token
%cd ~/stable-diffusion-webui

!python launch.py --listen --xformers --enable-insecure-extension-access --gradio-queue --ngrok 2XWKyblB2vN0oonn4F9owPN1gfL_4wZ4rxXdb8sPqsdfgW